# Text to SQL Query Translator

In this notebook we will use [Defog AI's model](https://huggingface.co/defog/llama-3-sqlcoder-8b) which is trained on llama 3 to translate natural language into SQL queries. The process can be divided into the parts mentioned below.


1) Creating a database engine. The engine will query data from the database and show us the results.

2) Importing the pretrained model and writing the prompt.

3) Asking questions in natural language for which the model will generating resposnes.

4) Cleaning the response and running the query on the engine toobtain results.


## 1) Importing libraries and setting up the environment

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install torch transformers bitsandbytes accelerate sqlparse &> /dev/null

In [3]:
import time
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from sqlalchemy import create_engine, inspect
import sqlparse


In [4]:
torch.cuda.is_available()

True

In [5]:
available_memory = torch.cuda.get_device_properties(0).total_memory
print(available_memory)

17059545088


## 2) Creating the database engine that will query and show us the results

In [6]:
# Create a database engine
engine = create_engine('sqlite:////kaggle/input/raw-transactional-data/transactional_data.sqlite')
# Context manager is used to establish a connection
with engine.connect() as conn:
    # SQLAlchemy's Inspector is used to retrieve table names
    inspector = inspect(engine)
    table_names = inspector.get_table_names()
# Print the table names
print(table_names)

['order_items', 'order_reviews', 'orders', 'products', 'sellers']


## 3) Imorting pre-trained model

In [7]:
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## 4) Prompt set up

In [8]:
#Setting up the schema of our database this will be used ahead in the prompt
schema="""

sellers Table:

seller_id: Unique identifier for each seller. Primary Key.
seller_state: State the seller belongs to.

orders Table:

order_id: Unique identifier for each order. Primary Key.
timestamp: Time when the order was placed.
customer_contact: Contact of the customer.

products Table:

product_id: Unique identifier for each product. Primary Key.
product_category_name: Category of the product.
product_weight_g: Weight of the product in grams.


order_items Table:

order_items_pk: Unique identifier for a row in this table. Primary Key.
order_id: Foreign key referencing the order to which the item belongs.
product_id: Foreign key referencing the order to which the product belongs.
seller_id: Foreign key referencing the order to which the seller belongs.
price: Price at which the product was sold.
The order_id column is a foreign key that references the order_id column in the orders table. This shows us details from a specific order. You can join this table with the orders table on order_id.
The product_id column is a foreign key that references the product_id column in the products table. This establishes a relationship between products and orders. You can join this table with the products table on product_id.
The seller_id column is a foreign key that references the seller_id column in the sellers table. This establishes a relationship between sellers and orders. You can join this table with the sellers table on seller_id.

order_reviews Table:

review_id: Unique identifier for each review. Primary Key.
order_id: Foreign key referencing the order to which the review belongs.
review_score: Score given by the customer.
review_comment_message: Contents of the review.
The order_id column is a foreign key that references the order_id column in the orders table. This establishes a relationship between orders and reviews. You can join this table with the orders table on order_id.

Potential joins:

If you want to analyze price of items along with their seller, you can join the order_items table with the sellers table using the seller_id column.
If you want to analyze price of items along with their category, you can join the order_items table with the products table using the product_id column.

"""

In [9]:
#This function takes a question and generates a prompt
def prompt_generator(question):
    prompt = f"""### Task
Generate a T-SQL query to answer the following question:

[QUESTION]
{question}
[/QUESTION]

### Instructions
-Leverage your expertise in MSSQL to generate efficient T-SQL queries that provide the needed data.
-Refer to the provided schema to determine the correct tables and columns for your query.
-For queries involving terms like "today" or "current," use the GETDATE() function to obtain the current date and time.
-Apply joins between tables as necessary to compile the relevant information.
-Limit your results to [top_k] rows using the OFFSET FETCH clause to enhance performance. You can adjust [top_k] when needed.
-Select only the specific columns required to answer the question, enclosing each column name in square brackets ([]) as delimited identifiers.
-Avoid selecting all columns from a table to improve efficiency.
-If the question cannot be resolved using the available schema, respond with 'I do not know'.
-When sorting results, manage NULL values carefully, and refrain from using the 'NULLS LAST' syntax.

### Database Schema
{schema}

**Missing Schema Handling:**
If the database schema information is missing, please provide it for query generation. Otherwise, this process will return 'Missing database schema'.

### Additional Information
- The database schema comprises multiple tables, each serving a distinct purpose.
- Ensure your SQL query is optimized for performance to handle large datasets effectively.

### sample examples :

question - "Which seller has the highest sales?"
result -"'SELECT s.seller_id,SUM(price) AS total_sales FROM sellers s JOIN order_items oi ON s.seller_id = oi.seller_id GROUP BY s.seller_id ORDER BY 2 DESC LIMIT 1;' "
            


### Answer
Based on the provided database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]:

[SQL]
"""
    return (prompt)

## 4) Query generation and cleaning

In [10]:
#This function takes a question calls the generate prompt function passes the prompt to the model and returns the response of the model
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [11]:
#This function will clean the response and keep only the query part so that we can run it using the engine
def clean_query(text):
    semicolon_pos = text.find(';')
    if semicolon_pos != -1:
        sql_query = text[:semicolon_pos + 1]
    else:
        sql_query = text
        
    cleaned_sql_query = sql_query.replace('\n', ' ').strip()

    return(cleaned_sql_query)

## 5) Viewing the results

In [12]:
question = 'What are the top 3 heaviest products?'
prompt = prompt_generator(question)
generated_sql = generate_query(question)
cleaned_query = clean_query(generated_sql)
result = pd.read_sql_query(cleaned_query, engine, coerce_float=False)
display(result)

2024-08-14 03:03:06.959215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 03:03:06.959340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 03:03:07.109189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,product_id,product_category_name,product_weight_g
0,26644690fde745fc4654719c3904e1db,cama_mesa_banho,40425.0
1,d0877f0094337c414d23f5a3c7bad20c,moveis_escritorio,30000.0
2,53f92b0474f91fcb5bd188c6a8075c38,utilidades_domesticas,30000.0


In [13]:
question = 'Which seller sold the most items?'
prompt = prompt_generator(question)
generated_sql = generate_query(question)
cleaned_query = clean_query(generated_sql)
result = pd.read_sql_query(cleaned_query, engine, coerce_float=False)
display(result)

,seller_id,total_items_sold
0,6560211a19b47992c3666cc44a7e94c0,1996


In [14]:
question = 'Which seller earned the most money?'
prompt = prompt_generator(question)
generated_sql = generate_query(question)
cleaned_query = clean_query(generated_sql)
result = pd.read_sql_query(cleaned_query, engine, coerce_float=False)
display(result)

,seller_id,total_sales
0,4869f7a5dfa277a7dca6462dcf3b52b2,226987.93


In [15]:
question = 'Which state has the most sellers?'
prompt = prompt_generator(question)
generated_sql = generate_query(question)
cleaned_query = clean_query(generated_sql)
result = pd.read_sql_query(cleaned_query, engine, coerce_float=False)
display(result)

,seller_state,seller_count
0,SP,1769


In [16]:
question = 'Which is the most popular category?'
prompt = prompt_generator(question)
generated_sql = generate_query(question)
cleaned_query = clean_query(generated_sql)
result = pd.read_sql_query(cleaned_query, engine, coerce_float=False)
display(result)

,product_category_name,product_count
0,cama_mesa_banho,2991
